In [ ]:
import pandas as pd
import os 
from google.cloud import bigquery
from google.cloud import storage 

def tree(): 
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="*******"
    
    tree = 'https://en.wikipedia.org/wiki/Rockefeller_Center_Christmas_Tree'
    tree_wiki = pd.read_html(tree, attrs={'class': 'wikitable'})
    print('Extracted {num} wikitables.'.format(num = len(tree_wiki)))
    df = tree_wiki[0]
    df['Original location'] = df['Original location'].fillna('No location provided')
    df['Lighting ceremony/Misc'] = df['Lighting ceremony/Misc'].fillna('No information provided')
    for column in df.columns:
        df['Original location'] = df['Original location'].str.replace(r'\[.*\]', ' ')
        df['Height'] = df['Height'].str.replace(r'\[.*\]', ' ')
    df = df.drop('Lighting ceremony/Misc', axis=1)
    df.rename(columns={'Original location': 'Original_location', 'Tree type': 'Tree_type'}, inplace=True)
    
    client = bigquery.Client()
    dataset_id = '******.xmas'
    table_id = 'xmas.tree'
    
    dataset_ref = client.dataset(dataset_id)
    dataset_table_id = dataset_ref.table(table_id)
    
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition='WRITE_TRUNCATE'
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect=True
    job_config.ignore_unknown_values=True 

    job = client.load_table_from_dataframe(
    df,
    table_id,
    location='US',
    job_config=job_config)
    
    job.result()
    return print('The job has loaded. Happy holidays!')

tree()
